In [ ]:
# !pip install bertopic
# !pip install sentence-transformers

In [ ]:
import pandas as pd

from google.colab import drive
import os
import shutil
import zipfile

drive.mount("/content/gdrive/") # 지정 위치와 구글 드라이브 연결

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
file_path = '/content/gdrive/MyDrive/GMSW/project/data/'
df = pd.read_csv(file_path+'all_news_data_ssk.csv')
df.tail()

## 토픽모델링

### 모델학습

In [ ]:
texts = df['content'].tolist()

# 불용어 리스트
my_stopwords = [
    '있다', '이날', '밝혔다', '따르면', '기자', '오전', '오후', '전일', '대비',
    '거래', '마감', '기록', '관련', '위해', '통해', '가장', '것으로', '보인다',
    '코스피', '포인트', '증권', '종목', '상승', '하락'
]

embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
vectorizer_model = CountVectorizer(stop_words=my_stopwords)

# 모델 생성 (min_topic_size 상향)
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model, # 불용어 적용
    min_topic_size=50,      # 50 (너무 작은 토픽 생성 방지)
    nr_topics=20,           # 자동으로 20개까지만 남기고 합치기
    top_n_words=10,
    verbose=True
)

# 학습
topics, probs = topic_model.fit_transform(texts)

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
# 토픽 간 코사인 유사도 히트맵
# 특정 토픽끼리 뭉치는 경향을 파악할 수 있음
fig2 = topic_model.visualize_heatmap()

# 만약 토픽이 너무 많다면, 주요 토픽만 골라서 그릴 수도 있음
# fig2 = topic_model.visualize_heatmap(topics_to_ignore=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]) # 노이즈 토픽 제외
fig2.show()

In [ ]:
# 토픽들의 계층적 구조 시각화 (Dendrogram)
# 어떤 토픽들이 같은 그룹으로 묶이는지 확인
fig3 = topic_model.visualize_hierarchy()
fig3.show()

In [ ]:
df['topic'] = topics
df[['date', 'content', 'topic']].head()

### save

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/GMSW/project/result/news_topic_df_ssk.csv')

In [ ]:
# # 모델저장
# save_path = "/content/gdrive/MyDrive/GMSW/project/model/news_topic_model_ssk"
# topic_model.save(save_path, serialization="safetensors")

In [ ]:
embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [ ]:
# # 저장했던 모델 불러오기
# save_path = "/content/gdrive/MyDrive/GMSW/project/model/news_topic_model_ssk"
# topic_model = BERTopic.load(save_path)

# # 잘 불러와졌는지 확인 (토픽 정보 출력)
# topic_model.get_topic_info()

### 토픽 레이블 지정


In [ ]:
# df[['topic']].value_counts()

In [ ]:
# # 1. 아웃라이어(Topic -1) 줄이기
# # strategy="c-tf-idf" 등을 쓰면 문맥상 가장 가까운 토픽으로 재배치해줍니다.
# new_topics = topic_model.reduce_outliers(texts, topics, strategy="c-tf-idf")

# # 2. 줄어든 결과로 토픽 업데이트
# topic_model.update_topics(texts, topics=new_topics)

# # 3. 다시 정보 확인 (Topic -1의 개수가 확 줄었는지 확인)
# topic_model.get_topic_info()

In [ ]:
# # 모델저장
# save_path = "/content/gdrive/MyDrive/GMSW/project/model/news_topic_model_ssk-reduce_outliers"
# topic_model.save(save_path, serialization="safetensors")

In [ ]:
topic_labels = {
    -1: "노이즈",
    0: "시황/랠리",
    1: "산업트렌드/실적/펀더멘털",
    2: "협력 기대감",
    3: "글로벌 거시 정치",
    4: "정책",
    5: "이벤트",
    6: "경영 승계/보상",
    7: "노이즈"
}

df['Topic_Name'] = df['topic'].map(topic_labels)

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/GMSW/project/result/news_topic_df_ssk.csv')

### 토픽-주가 비교 시각화

In [ ]:
# 저장했던 모델 불러오기
save_path = "/content/gdrive/MyDrive/GMSW/project/model/news_topic_model_ssk"
topic_model = BERTopic.load(save_path)

# 모델이 기억하고 있는 토픽 정보(topics_)
current_topics = topic_model.topics_

# 텍스트와 토픽 정보를 이용해 '단어 가중치(c-TF-IDF)' 다시 계산
topic_model.update_topics(df['content'].tolist(), topics=current_topics)

# 잘 불러와졌는지 확인 (토픽 정보 출력)
topic_model.get_topic_info()

In [ ]:
# 시계열 분석 계산
texts = df['content'].tolist()
timestamps = df['date'].tolist()

topics_over_time = topic_model.topics_over_time(texts, timestamps, nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=5)

In [ ]:
import datetime

topics_over_time['date'] = [time.strftime('%Y-%m-%d') for time in topics_over_time['Timestamp'].tolist()]
topics_over_time.head()

In [ ]:
stock = pd.read_csv(file_path+'stock_price.csv')
# stock.head()

In [ ]:
# Min-Max 정규화
def min_max_scale(series):
    return (series - series.min()) / (series.max() - series.min())

# 정규화된 컬럼 생성
stock['Scaled_samsung_price'] = min_max_scale(stock['samsung_price'])
stock['Scaled_skhynix_price'] = min_max_scale(stock['skhynix_price'])

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 토픽 데이터 준비 (topics_over_time 결과 사용)
target_topics = [0, 1, 2, 3]
topic_labels = {
    0: "Topic 0: 시황/랠리",
    1: "Topic 1: 산업/실적/펀더멘털",
    2: "Topic 2: 협력기대감",
    3: "Topic 3: 매크로",
}

# 이중축 차트 그리기
# 보조축(Secondary Y)이 있는 피규어 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])

# [왼쪽 축] 토픽 빈도수 (막대 그래프 or 부드러운 선)
colors = ['#FFA07A', '#20B2AA', '#87CEFA', '#fbe383']

for i, topic_id in enumerate(target_topics):
    # 해당 토픽 데이터만 필터링
    topic_data = topics_over_time[topics_over_time['Topic'] == topic_id]

    fig.add_trace(
        go.Scatter(
            x=topic_data['Timestamp'],
            y=topic_data['Frequency'],
            name=topic_labels[topic_id],
            mode='lines', # 'lines' 또는 'lines+markers' 추천
            line=dict(width=2, color=colors[i]),
            fill='tozeroy', # 바닥까지 색칠하기 (영역 차트 느낌)
            opacity=0.5     # 투명도 조절 (주가가 잘 보이게)
        ),
        secondary_y=False # 왼쪽 축 사용
    )

In [ ]:
# [오른쪽 축] 주가 데이터 (선 그래프)
# 삼성전자
fig.add_trace(
    go.Scatter(
        x=stock['date'],
        y=stock['Scaled_samsung_price'],
        name="삼성전자 (Scaled)",
        mode='lines',
        line=dict(color='firebrick', width=2, dash='dot') #, dash='solid')
    ),
    secondary_y=True # 오른쪽 축 사용
)

# SK하이닉스 (필요하면 주석 해제)
fig.add_trace(
    go.Scatter(
        x=stock['date'],
        y=stock['Scaled_skhynix_price'],
        name="SK하이닉스 (Scaled)",
        mode='lines',
        line=dict(color='royalblue', width=2, dash='dot') # 점선으로 구분
    ),
    secondary_y=True
)

# 레이아웃 다듬기
fig.update_layout(
    title_text="<b>뉴스 토픽 변화와 주가 추세의 상관관계 (Min-Max Scaled)</b>",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor='center'), # 범례 위로 올리기
    height=600
)

fig.update_yaxes(title_text="<b>토픽 언급 빈도 (Frequency)</b>", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="<b>주가 추세 강도 </b>", secondary_y=True, showgrid=True)

fig.show()

#### Granger Causality Test

In [ ]:
target_topic_id = 2
target_stock = stock[['date', 'skhynix_price']] # 분석할 주가: SK하이닉스

# 토픽 데이터 추출 (일별 빈도 합계)
topic_data = topics_over_time[topics_over_time['Topic'] == target_topic_id]
topic_daily = topic_data.set_index('date')['Frequency']
topic_daily

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

# ---------------------------------------------------------
# 1. 데이터 준비 (분석할 토픽과 주가를 선택하세요)
# ---------------------------------------------------------
target_topic_id = 4  # 분석할 토픽: Topic 1 (AI 트렌드)
target_stock = stock[['date', 'skhynix_price']] # 분석할 주가: SK하이닉스

# 토픽 데이터 추출 (일별 빈도 합계)
topic_data = topics_over_time[topics_over_time['Topic'] == target_topic_id]
topic_daily = topic_data.set_index('date')['Frequency']

# topic_daily = topic_daily.reindex(target_stock['date'], fill_value=0)

# 데이터 병합 (날짜 기준 교집합)
df_analysis = pd.merge(topic_daily, target_stock, on='date').dropna()
df_analysis.columns = ['Date', 'Topic_Freq', 'Stock_Price']

# ---------------------------------------------------------
# 2. 정상성(Stationarity) 검정 & 차분(Differencing)
# ---------------------------------------------------------
# 시계열 데이터는 계속 오르는 추세(Trend)가 있으면 통계 오류가 납니다.
# 따라서 '전일 대비 변화량(Change)'으로 변환해야 합니다.

def check_stationarity(series):
    result = adfuller(series)
    return result[1] # p-value 반환 (0.05 이하면 정상성 확보)

# 로그 차분 (수익률 개념) 적용
df_diff = np.log(df_analysis[['Topic_Freq', 'Stock_Price']]).diff().dropna()

print(f"토픽 빈도 정상성 p-value: {check_stationarity(df_diff['Topic_Freq']):.4f}")
print(f"주가 수익률 정상성 p-value: {check_stationarity(df_diff['Stock_Price']):.4f}")
# 둘 다 0.05 이하여야 다음 단계로 넘어갑니다!

In [ ]:
# ---------------------------------------------------------
# 3. 그레인저 인과관계 테스트
# ---------------------------------------------------------
# maxlag: 며칠 전 뉴스까지 볼 것인가? (보통 1주~2주 치인 5~10일 설정)
max_lag = 2

print(f"\n[검정 결과] Topic {target_topic_id} -> SK하이닉스 주가")
test_result = grangercausalitytests(df_diff[['Stock_Price', 'Topic_Freq']], maxlag=max_lag, verbose=True)

# 파라미터 순서: [종속변수(결과), 독립변수(원인)] -> 'Topic_Freq'가 'Stock_Price'를 선행하는지

In [ ]:
noise_topics = [0, 1, 2, 3, 4, 5]

df_clean = df[df['topic'].isin(noise_topics)].copy()
# df_clean = df_clean[df_clean['Sentiment_Score']>=0.5].copy()

print(f"원본 데이터: {len(df)}개 -> 정제된 데이터: {len(df_clean)}개")

# 이벤트 시작일 지정
event_start_date = "2025-10-16"

# 날짜 형식 변환
df_clean['date'] = pd.to_datetime(df_clean['date'])
threshold_date = pd.to_datetime(event_start_date)

# Pre: 이벤트 이전 / Post: 이벤트 이후 (급등 구간)
group_pre = df_clean[df_clean['date'] < threshold_date]
group_post = df_clean[df_clean['date'] >= threshold_date]

print(f"분석 기간 설정: {event_start_date} 기준")
print(f"Pre-Event 표본 수: {len(group_pre)}개")
print(f"Post-Event 표본 수: {len(group_post)}개")

In [ ]:
import plotly.express as px

# 시각화를 위한 히스토그램 (Histogram)
# barmode='overlay': 두 그래프를 겹쳐서 비교
# histnorm='probability density': 데이터 개수가 달라도 비율로 비교 가능하게 함

df_clean['Period'] = df_clean['date'].apply(
    lambda x: 'Post-Event' if x >= threshold_date else 'Pre-Event'
)
# 1. 기간(Pre/Post)별 토픽 비중 계산
topic_dist = df_clean.groupby(['Period', 'Topic_Name']).size().reset_index(name='Count')

# 2. 비율(%)로 변환
topic_dist['Percentage'] = topic_dist.groupby('Period')['Count'].transform(lambda x: x / x.sum())

# 3. 시각화 (100% Stacked Bar Chart)
fig = px.bar(topic_dist,
             x='Period',
             y='Percentage',
             color='Topic_Name',
             title="<b>이벤트 전후 뉴스 토픽 구성 변화 (Composition Shift)</b>",
             text_auto='.1%',
             category_orders={"Period": ["Pre-Event", "Post-Event"]} # 순서 고정
            )

fig.update_layout(
    yaxis_title="비중 (%)",
    template="plotly_white",
    legend=dict(orientation="h", y=1.1, x=0, xanchor='left'), # 범례 위로 올리기
)

fig.show()